# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2025</br>

Name: Thinh Le
</br>
Date: April 11, 2025
<br>
<br>
**At this time in the semester:** <br>
- We have explored a dataset. <br>
- We have cleaned our dataset. <br>
- We created a Github account with a repository for this class and included a metadata read me file about our data. <br>
- We introduced general SQL syntax, queries, and applications in Python.<br>
- Created our own databases from scratch using MySQL Workbench and Python with SQLAlchemy on our local server and locally on our machine.
- Populated our databases with the data we cleaned at the start of the semester.
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source, especially when posting your assignments to Github.<br>

Feel free to use your preferred library and method for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook!<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. Please submit this assignment with a link to it posted to your Github.<br>

**Data origin:** spotify database

## Load libraries

In [1]:
# Load necessary packages:
from sqlalchemy import create_engine, inspect, text
import pandas as pd # Python data manilpulation
from sqlalchemy.orm import Session

## Connect to database and create engine

In [2]:
# Create a SQLite database and engine
db_file = "spotify_data.db"
engine = create_engine(f"sqlite:///{db_file}")

## Create an inspector

In [3]:
inspector = inspect(engine)

Inspect all table names:

In [4]:
# Write a query to list the table names of the database:
tables = inspector.get_table_names()
print(tables)

['spotify_history']


Inspect all column names in the `spotify_history` table:

In [5]:
columns = inspector.get_columns('spotify_history')
print("All columns in spotify_history table:")
for col in columns:
    print(col["name"], "-", col["type"])

All columns in spotify_history table:
ts - TEXT
platform - TEXT
ms_played - BIGINT
conn_country - TEXT
ip_addr - TEXT
master_metadata_track_name - TEXT
master_metadata_album_artist_name - TEXT
master_metadata_album_album_name - TEXT
spotify_track_uri - TEXT
episode_name - FLOAT
episode_show_name - FLOAT
spotify_episode_uri - FLOAT
audiobook_title - FLOAT
audiobook_uri - FLOAT
audiobook_chapter_uri - FLOAT
audiobook_chapter_title - FLOAT
reason_start - TEXT
reason_end - TEXT
shuffle - BOOLEAN
skipped - BOOLEAN
offline - BOOLEAN
offline_timestamp - FLOAT
incognito_mode - BOOLEAN
year - BIGINT


View top 5 records from the table:

In [6]:
query = """SELECT * FROM spotify_history LIMIT 5"""
_5_records = pd.read_sql(query, engine)

_5_records

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,year
0,2012-08-03 15:43:50,OS X 10.7.4 [x86 4],137760,US,134.210.225.27,Some Nights - Intro,fun.,Some Nights,spotify:track:1JAI5Ia020mdGH2wMQEacy,None,...,None,None,uriopen,trackdone,0,0,0,None,0,2012
1,2012-08-03 15:48:28,OS X 10.7.4 [x86 4],277040,US,134.210.225.27,Some Nights,fun.,Some Nights,spotify:track:6t6oULCRS6hnI7rm0h5gwl,None,...,None,None,trackdone,trackdone,0,0,0,None,0,2012
2,2012-08-03 15:50:16,OS X 10.7.4 [x86 4],108244,US,134.210.225.27,We Are Young (feat. Janelle Monáe),fun.,Some Nights,spotify:track:7a86XRg84qjasly9f6bPSD,None,...,None,None,trackdone,uriopen,0,1,0,None,0,2012
3,2012-08-03 15:50:31,OS X 10.7.4 [x86 4],16015,US,134.210.225.27,Trip to Your Heart,Britney Spears,Femme Fatale (Deluxe Version),spotify:track:2qbhijQG7phGVHkPt22fTP,None,...,None,None,uriopen,uriopen,0,1,0,None,0,2012
4,2012-08-03 15:53:53,OS X 10.7.4 [x86 4],73786,US,134.210.225.27,Stan,Eminem,The Marshall Mathers LP,spotify:track:3UmaczJpikHgJFyBTAJVoz,None,...,None,None,uriopen,popup,0,1,0,None,0,2012


#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** Which songs played within 5 years ago? Which songs played more than 5 years ago?
<br>
**What tables are we joining? (If any):** I won't do the join since there is only 1 table in the dataset

In [7]:
# Which songs played within 5 years ago? Which songs played more than 5 years ago?
# strftime: Extract year from string
query = """
SELECT
    master_metadata_track_name AS song_name,
    CASE
        WHEN CAST(strftime('%Y', ts) AS INTEGER) >= 2020 THEN 'Within the past 5 years'
        ELSE 'From 5 years ago and before'
    END AS time_range
FROM spotify_history
"""
year_check = pd.read_sql(query, engine)

print(year_check)

                                        song_name                   time_range
0                             Some Nights - Intro  From 5 years ago and before
1                                     Some Nights  From 5 years ago and before
2              We Are Young (feat. Janelle Monáe)  From 5 years ago and before
3                              Trip to Your Heart  From 5 years ago and before
4                                            Stan  From 5 years ago and before
...                                           ...                          ...
202673  Red - From "Descendants: The Rise of Red"      Within the past 5 years
202674                            Life Is Sweeter      Within the past 5 years
202675                              Space Between      Within the past 5 years
202676                            Perfect Revenge      Within the past 5 years
202677                             Back to Autumn      Within the past 5 years

[202678 rows x 2 columns]


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Example Question:** Count the number of different devices (e.g., Android, iOS, and others) used
<br>
**What tables are we joining? (If any):** I won't do the join since there is only 1 table in the dataset

In [8]:
# Count the number of different devices (e.g., Android, iOS, and others) used
query = """
SELECT
    platform,
    COUNT(*) AS platform_count,
    CASE
        WHEN LOWER(platform) LIKE '%android%' THEN 'Android platform'
        WHEN LOWER(platform) LIKE '%ios%' THEN 'iOS platform'
        WHEN LOWER(platform) LIKE 'os%' THEN 'OS platform'
        WHEN LOWER(platform) LIKE '%google%' THEN 'Google platform'
        ELSE 'Other'
    END AS platform_check
FROM spotify_history
GROUP BY platform;
"""
platform_check = pd.read_sql(query, engine)

platform_check

,platform,platform_count,platform_check
0,OS X 10.7.4 [x86 4],70,OS platform
1,OS X 10.7.5 [x86 4],3604,OS platform
2,OS X 10.9.0 [x86 4],753,OS platform
3,OS X 10.9.1 [x86 4],596,OS platform
4,OS X 10.9.2 [x86 4],856,OS platform
...,...,...,...
176,"iOS 9.3.2 (iPhone6,1)",631,iOS platform
177,"iOS 9.3.5 (iPhone6,1)",8019,iOS platform
178,ios,38659,iOS platform
179,osx,205,OS platform


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** Count songs has `ms_played` under 1 minute, from 1 and under 3 minutes, from 3 minutes or above?
<br>
**What tables are we joining? (If any):** I won't do the join since there is only 1 table in the dataset

In [22]:
# Count songs has `ms_played` under 1 minute, from 1 and under 3 minutes, from 3 minutes or above?
# < 60000: Under 1 minute
# between 60000 and 180000: From 1 minute and under 3 minutes
# else: 'From 3 minutes or above'
query = """
SELECT
    CASE
        WHEN ms_played < 60000 THEN 'Under 1 minute'
        WHEN ms_played >= 60000 AND ms_played < 180000 THEN 'From 1 minute and under 3 minutes'
        ELSE 'From 3 minutes or above'
    END AS ms_played_check,
    COUNT(*) AS ms_played_count
FROM spotify_history
GROUP BY ms_played_check;
"""
ms_played_check = pd.read_sql(query, engine)

ms_played_check

,ms_played_check,ms_played_count
0,From 1 minute and under 3 minutes,34543
1,From 3 minutes or above,61798
2,Under 1 minute,106338


**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Question:**  Count the number of tracks which were played online
<br>

In [10]:
# Count the number of tracks which were played online
query = """
SELECT COUNT(*)
FROM spotify_history
WHERE offline_timestamp IS NULL;
"""
online_track_check = pd.read_sql(query, engine)

online_track_check

,COUNT(*)
0,162288


**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Insert a new record to the database

In [13]:
# Open a session and insert a new record
with Session(engine) as session:
    query = """
    INSERT INTO spotify_history
    (ts, master_metadata_track_name, master_metadata_album_artist_name)
    VALUES ('2025-04-12 11:18:50', 'everything, everywhere', 'vaultboy')"""
    session.execute(text(query))
    session.commit()

Check the inserted value:

In [15]:
query = """
SELECT * FROM spotify_history
WHERE master_metadata_track_name = 'everything, everywhere';
"""
insert_track_check = pd.read_sql(query, engine)

insert_track_check

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,year
0,2025-04-12 11:18:50,None,None,None,None,"everything, everywhere",vaultboy,None,None,None,...,None,None,None,None,None,None,None,None,None,None


**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above.<br>
The IN operator in a WHERE clause lets you filter for multiple values at once. You can also exclude certain values by using the NOT IN operator.
<br>

**Question:** Find the max `ms_played`

In [16]:
# Find the max `ms_played`
query = """
SELECT MAX(ms_played) FROM spotify_history
"""
max_ms_played = pd.read_sql(query, engine)

max_ms_played

,MAX(ms_played)
0,3570734


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** Find min and max `ms_played` from artists and assign CASE definitions for Listener Categories

In [19]:
# Find min and max `ms_played` from artists and assign CASE definitions for Listener Categories
# Less than 1 year: Recently listened to
# From 1 to 5 years: Quite a long time fan
# More than 5 years: Must be a very loyal fan!
query = """
SELECT
    master_metadata_album_artist_name,
    MIN(year) AS min_year,
    MAX(year) AS max_year,
    CASE
        WHEN MAX(year) - MIN(year) < 1 THEN 'Recently listened to'
        WHEN MAX(year) - MIN(year) BETWEEN 1 AND 5 THEN 'Quite a long time fan'
        ELSE 'Must be a very loyal fan!'
    END AS year_check
FROM spotify_history
GROUP BY master_metadata_album_artist_name
ORDER BY year_check ASC;
"""
listener_category_check = pd.read_sql(query, engine)

listener_category_check

,master_metadata_album_artist_name,min_year,max_year,year_check
0,'Legally Blonde' Ensemble,2016,2023,Must be a very loyal fan!
1,*NSYNC,2012,2024,Must be a very loyal fan!
2,2 Chainz,2012,2024,Must be a very loyal fan!
3,21 Savage,2016,2024,Must be a very loyal fan!
4,2Pac,2012,2024,Must be a very loyal fan!
...,...,...,...,...
6031,Ásgeir,2017,2017,Recently listened to
6032,Édith Piaf,2013,2013,Recently listened to
6033,Ólafur Arnalds,2020,2020,Recently listened to
6034,Öwnboss,2024,2024,Recently listened to


**MIN() and MAX()** <br>
AVG() will take the average of a numeric field.

**Question:** Find the average `ms_played` time
<br>

In [21]:
# Find the average `ms_played` time
query = """
SELECT AVG(ms_played) FROM spotify_history
"""
avg_ms_played = pd.read_sql(query, engine)

avg_ms_played

,AVG(ms_played)
0,98223.767345


Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above. <br><br>
**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above (If you have not done so already). Be sure to obtain the same result set.

In [23]:
query = """
SELECT
    sh.master_metadata_track_name AS song_name,
    AVG(sh.ms_played) AS average_ms_played
FROM spotify_history sh
GROUP BY sh.master_metadata_track_name
"""
avg_ms_played_v2 = pd.read_sql(query, engine)
avg_ms_played_v2

,song_name,average_ms_played
0,!!!!!!!,6773.565217
1,"""Impossible""",1088.000000
2,"""In My Own Little Corner""",853.000000
3,"""Stepsister's Lament""",13861.000000
4,"""The Take Over, The Breaks Over""",20912.000000
...,...,...
21746,¡Hola Casita!,6878.250000
21747,Âme Douce,41633.000000
21748,Échame La Culpa,72100.235294
21749,Ödla Resa,138571.000000


Now we are starting to create multiple new fields that we can save any time as a .csv if needed to access later. Save your result set as a .csv:

In [24]:
# Save result set as .csv file:
avg_ms_played_v2.to_csv('average-ms-played-by-song.csv', index=False)

**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Objective:** Delete the record that I inserted above

In [25]:
# Delete the record that I inserted above
# Open a session and delete the previous added record
with Session(engine) as session:
    query = """
    DELETE FROM spotify_history
    WHERE master_metadata_track_name = 'everything, everywhere'
    AND ts = '2025-04-12 11:18:50'"""
    session.execute(text(query))
    session.commit()

Try to search for the record again and expect to see no results:

In [27]:
query = """
SELECT * FROM spotify_history
WHERE master_metadata_track_name = 'everything, everywhere'
AND ts = '2025-04-12 11:18:50'
"""
deleted_track_check = pd.read_sql(query, engine)

deleted_track_check

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,year


We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>

In [28]:
#Close the database connection :)
session.close()
engine.dispose()

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. It is preferred that you submit your Github link instead of the file itself.